In [1]:
import pandas as pd
import pdb
import numpy as np
import itertools
from utils import *

## 230516 metadata + file name integration

In [2]:
fname = '230516_config.tsv'
datasets_per_run = 7
auto_dedupe = True
meta_fname = 'mouse_metadata.tsv'

In [3]:
df, dataset_df = parse_config_file(fname,
                      meta_fname,
                      datasets_per_run,
                      auto_dedupe=True)

In [4]:
print(len(df.index))
print(len(dataset_df.index))

56
29


In [7]:
print(len(dataset_df.dataset.unique()))
dataset_df.head()

29


,sample,mouse_id,genotype,sex,age,tissue,biorep_num,dataset,talon_run_num
0,5xFADWT_M_18_months_CTX,10721,5xFADWT,M,18_months,CTX,1,5xFADWT_M_18_months_CTX_1,1
2,5xFADHEMI_M_18_months_CTX,10725,5xFADHEMI,M,18_months,CTX,1,5xFADHEMI_M_18_months_CTX_1,1
4,5xFADHEMI_F_4_months_HC,11616,5xFADHEMI,F,4_months,HC,1,5xFADHEMI_F_4_months_HC_1,1
6,5xFADHEMI_F_4_months_HC,11617,5xFADHEMI,F,4_months,HC,2,5xFADHEMI_F_4_months_HC_2,1
8,5xFADHEMI_F_4_months_HC,11625,5xFADHEMI,F,4_months,HC,3,5xFADHEMI_F_4_months_HC_3,1


In [6]:
df.head()

,path,basename,batch,platform,fname,flowcell,file_stem,chop_num,sample_temp,mouse_id,genotype,sex,age,tissue,sample,biorep_num,dataset,talon_run_num
0,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-blk_2_t1.fastq.gz,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10721_lig-blk_2,1.0,ad002_10721,10721,5xFADWT,M,18_months,CTX,5xFADWT_M_18_months_CTX,1,5xFADWT_M_18_months_CTX_1,NaN
1,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-ss_blk_1_t1.fastq.gz,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10721_lig-ss_blk_1,1.0,ad002_10721,10721,5xFADWT,M,18_months,CTX,5xFADWT_M_18_months_CTX,1,5xFADWT_M_18_months_CTX_1,NaN
2,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-blk_2_t1.fastq.gz,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10725_lig-blk_2,1.0,ad002_10725,10725,5xFADHEMI,M,18_months,CTX,5xFADHEMI_M_18_months_CTX,1,5xFADHEMI_M_18_months_CTX_1,NaN
3,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-ss_blk_1_t1.fastq.gz,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10725_lig-ss_blk_1,1.0,ad002_10725,10725,5xFADHEMI,M,18_months,CTX,5xFADHEMI_M_18_months_CTX,1,5xFADHEMI_M_18_months_CTX_1,NaN
4,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,230516,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,5xFADHEMI,F,4_months,HC,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1,NaN


In [37]:
# """
# Parameters:
#     fname (str): Path to config file fname. One line per input fastq.
#     meta_fname (str): Path to file with metadata information.
#     datasets_per_run (int): Number of datasets to process in each TALON run
#     auto_dedupe (bool): Automatically deduplicate duplicate fastqs that result from 
#         successive Porechop rounds

# Returns:
#     df (pandas DataFrame): DF w/ pipeline information; one line per fastq
#     dataset_df (pandas DataFrame): DF w/ dataset information; one line per mouse
# """

# df = pd.read_csv(fname, sep='\t')

# ############ Dataset + flowcell df

# # get flowcell
# exp = '.*\/[\w-]+_(\d+)(?:_t\d+)?\.fastq(?:.gz)?'
# df['flowcell'] = df.fname.str.extract(exp)

# # check to make sure the same file stem isn't there more than once
# # (can happen if different flow cells needed different amounts of chopping)
# # df['file_stem'] = df.basename.str.rsplit('_', n=1, expand=True)[0]
# exp = '.*\/([\w-]+_\d+)(?:_t\d+)?\.fastq(?:.gz)?'
# df['file_stem'] = df.fname.str.extract(exp)
# df['chop_num'] = df.basename.str.rsplit('.fastq', expand=True)[0].str.rsplit('_t', expand=True)[1].astype(float)
# if df.file_stem.duplicated().any():
#     dupe_stems = df.loc[df.file_stem.duplicated(keep=False), 'basename'].tolist()
#     if not auto_dedupe:
#         raise ValueError(f'Files {dupe_stems} seem to be duplicated. Check config file.')
#     else:
#         print(f'Files {dupe_stems} seem to be duplicated. Automatically removing lower chop numbers')
#         df = df.sort_values(by='chop_num', ascending=False)
#         df = df.drop_duplicates(subset='file_stem', keep='first')

# # extract the sample name
# temp = df.basename.str.split('_', expand=True)[[0,1]]#.str.join('_')
# df['sample_temp'] = temp[0]+'_'+temp[1]

# # extract the mouse id
# df['mouse_id'] = df['sample_temp'].str.split('_', expand=True)[1]

# # merge in metadata
# meta = process_meta(meta_fname)
# df['mouse_id'] = df['mouse_id'].astype('int')
# df = df.merge(meta, how='left', on='mouse_id')

# # get tech rep numbers -- each mouse has multiple reps
# # and are therefore technical reps
# df['flowcell'] = df.sort_values(['genotype', 'mouse_id'],
#                             ascending=[True, True])\
#                             .groupby(['mouse_id']) \
#                             .cumcount() + 1

# # sample should be the genotype + age + sex + tissue
# df['sample'] = df.genotype+'_'+ \
#                df.sex+'_'+ \
#                df.age+'_'+ \
#                df.tissue

# # get biorep numbers -- each mouse_id is a different mouse
# # and therefore a different biorep
# temp = df[['sample', 'mouse_id']].drop_duplicates()
# temp.reset_index(inplace=True, drop=True)
# temp['biorep_num'] = temp.sort_values(['sample', 'mouse_id'],
#                             ascending=[True, True])\
#                             .groupby(['sample']) \
#                             .cumcount()+1
# df = df.merge(temp, how='left',
#               on=['sample', 'mouse_id'])

# # talon dataset should be sample + bio rep
# df['talon_dataset'] = df['sample']+'_'+df['biorep_num'].astype(str)

# # dataset should be sample + bio rep + flow cel
# df['dataset'] = df['talon_dataset']+'_'+df['flowcell'].astype(str)

# ############ TALON dataset df

# # create a dataset-level df that will represent the aggregate
# cols = ['sample', 'mouse_id', 'genotype', 'sex', \
#         'age', 'tissue', 'biorep_num', 'talon_dataset']
# dataset_df = df[cols].drop_duplicates()

# # get the talon run number these will go into
# talon_run_num = 0
# df['talon_run_num'] = np.nan
# for ind, entry, in dataset_df.iterrows():
#     if ind % datasets_per_run == 0:
#         talon_run_num += 1
#     dataset_df.loc[ind, 'talon_run_num'] = talon_run_num
# dataset_df['talon_run_num'] = dataset_df.talon_run_num.astype(int)

In [36]:
dataset_df

,sample,mouse_id,genotype,sex,age,tissue,biorep_num,talon_dataset,talon_run_num
0,5xFADWT_M_18_months_CTX,10721,5xFADWT,M,18_months,CTX,1,5xFADWT_M_18_months_CTX_1,1
2,5xFADHEMI_M_18_months_CTX,10725,5xFADHEMI,M,18_months,CTX,1,5xFADHEMI_M_18_months_CTX_1,1
4,5xFADHEMI_F_4_months_HC,11616,5xFADHEMI,F,4_months,HC,1,5xFADHEMI_F_4_months_HC_1,1
6,5xFADHEMI_F_4_months_HC,11617,5xFADHEMI,F,4_months,HC,2,5xFADHEMI_F_4_months_HC_2,1
8,5xFADHEMI_F_4_months_HC,11625,5xFADHEMI,F,4_months,HC,3,5xFADHEMI_F_4_months_HC_3,1
10,5xFADWT_F_4_months_HC,11627,5xFADWT,F,4_months,HC,1,5xFADWT_F_4_months_HC_1,1
12,5xFADWT_F_4_months_HC,11628,5xFADWT,F,4_months,HC,2,5xFADWT_F_4_months_HC_2,1
14,5xFADWT_F_4_months_HC,11629,5xFADWT,F,4_months,HC,3,5xFADWT_F_4_months_HC_3,2
16,5xCLU-h2kbKI_HO_F_4_months_HC,12517,5xCLU-h2kbKI_HO,F,4_months,HC,1,5xCLU-h2kbKI_HO_F_4_months_HC_1,2
18,CLU-h2kbKI_HO_F_4_months_HC,12648,CLU-h2kbKI_HO,F,4_months,HC,1,CLU-h2kbKI_HO_F_4_months_HC_1,2


In [45]:
def process_meta(meta_fname):
    meta = pd.read_csv(meta_fname, sep='\t')
    meta['genotype'] = meta.genotype.str.replace(' ', '_')
    meta['genotype'] = meta.genotype.str.replace('/', '_')

    meta['age'] = meta.age.str.replace(' ', '_')
    
    return meta

In [55]:
df = pd.read_csv(fname, sep='\t')

# get flowcell
exp = '.*\/[\w-]+_(\d+)(?:_t\d+)?\.fastq(?:.gz)?'
df['flowcell'] = df.fname.str.extract(exp)

# check to make sure the same file stem isn't there more than once
# (can happen if different flow cells needed different amounts of chopping)
# df['file_stem'] = df.basename.str.rsplit('_', n=1, expand=True)[0]
exp = '.*\/([\w-]+_\d+)(?:_t\d+)?\.fastq(?:.gz)?'
df['file_stem'] = df.fname.str.extract(exp)
df['chop_num'] = df.basename.str.rsplit('.fastq', expand=True)[0].str.rsplit('_t', expand=True)[1].astype(float)
if df.file_stem.duplicated().any():
    dupe_stems = df.loc[df.file_stem.duplicated(keep=False), 'basename'].tolist()
    if not auto_dedupe:
        raise ValueError(f'Files {dupe_stems} seem to be duplicated. Check config file.')
    else:
        print(f'Files {dupe_stems} seem to be duplicated. Automatically removing lower chop numbers')
        df = df.sort_values(by='chop_num', ascending=False)
        df = df.drop_duplicates(subset='file_stem', keep='first')

# extract the sample name (?)
temp = df.basename.str.split('_', expand=True)[[0,1]]#.str.join('_')
df['sample_temp'] = temp[0]+'_'+temp[1]

# extract the mouse id
df['mouse_id'] = df['sample_temp'].str.split('_', expand=True)[1]

# merge in metadata
meta = process_meta(meta_fname)
df['mouse_id'] = df['mouse_id'].astype('int')
df = df.merge(meta, how='left', on='mouse_id')

# get tech rep numbers -- each mouse has multiple reps
# and are therefore technical reps
df['techrep_num'] = df.sort_values(['genotype', 'mouse_id'],
                            ascending=[True, True])\
                            .groupby(['mouse_id']) \
                            .cumcount() + 1

# sample should be the genotype + age + sex + tissue
df['sample'] = df.genotype+'_'+ \
               df.sex+'_'+ \
               df.age+'_'+ \
               df.tissue

# get biorep numbers -- each mouse_id is a different mouse
# and therefore a different biorep
temp = df[['sample', 'mouse_id']].drop_duplicates()
temp.reset_index(inplace=True, drop=True)
temp['biorep_num'] = temp.sort_values(['sample', 'mouse_id'],
                            ascending=[True, True])\
                            .groupby(['sample']) \
                            .cumcount()+1
df = df.merge(temp, how='left',
              on=['sample', 'mouse_id'])

# dataset should be sample + bio rep + tech rep
df['dataset'] = df['sample']+'_'+df.biorep_num.astype(str)+'_'+df.techrep_num.astype(str)

# get the talon run number these will go into
talon_run_num = 0
df['talon_run_num'] = np.nan
for ind, entry, in df.iterrows():
    if ind % datasets_per_run == 0:
        talon_run_num += 1
    df.loc[ind, 'talon_run_num'] = talon_run_num
df['talon_run_num'] = df.talon_run_num.astype(int)

In [56]:
df

,path,basename,batch,platform,fname,flowcell,file_stem,chop_num,sample_temp,mouse_id,genotype,sex,age,tissue,techrep_num,sample,biorep_num,dataset,talon_run_num
0,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-blk_2_t1.fastq.gz,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10721_lig-blk_2,1.0,ad002_10721,10721,5xFADWT,M,18_months,CTX,1,5xFADWT_M_18_months_CTX,1,5xFADWT_M_18_months_CTX_1_1,1
1,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-ss_blk_1_t1.fastq.gz,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10721_lig-ss_blk_1,1.0,ad002_10721,10721,5xFADWT,M,18_months,CTX,2,5xFADWT_M_18_months_CTX,1,5xFADWT_M_18_months_CTX_1_2,1
2,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-blk_2_t1.fastq.gz,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10725_lig-blk_2,1.0,ad002_10725,10725,5xFADHEMI,M,18_months,CTX,1,5xFADHEMI_M_18_months_CTX,1,5xFADHEMI_M_18_months_CTX_1_1,1
3,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-ss_blk_1_t1.fastq.gz,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10725_lig-ss_blk_1,1.0,ad002_10725,10725,5xFADHEMI,M,18_months,CTX,2,5xFADHEMI_M_18_months_CTX,1,5xFADHEMI_M_18_months_CTX_1_2,1
4,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,11616,5xFADHEMI,F,4_months,HC,1,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1_1,1
5,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_2_t1.fastq,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11616_lig-blk_2,1.0,ad003_11616,11616,5xFADHEMI,F,4_months,HC,2,5xFADHEMI_F_4_months_HC,1,5xFADHEMI_F_4_months_HC_1_2,1
6,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,11617,5xFADHEMI,F,4_months,HC,1,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2_1,1
7,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_2_t1.fastq,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11617_lig-blk_2,1.0,ad003_11617,11617,5xFADHEMI,F,4_months,HC,2,5xFADHEMI_F_4_months_HC,2,5xFADHEMI_F_4_months_HC_2_2,2
8,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_1_t2.fastq,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11625_lig-blk_1,2.0,ad003_11625,11625,5xFADHEMI,F,4_months,HC,1,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3_1,2
9,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_2_t1.fastq,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11625_lig-blk_2,1.0,ad003_11625,11625,5xFADHEMI,F,4_months,HC,2,5xFADHEMI_F_4_months_HC,3,5xFADHEMI_F_4_months_HC_3_2,2


In [47]:
# def parse_config_file(fname,
#                       datasets_per_run,
#                       auto_dedupe=True):

#     df = pd.read_csv(fname, sep='\t')

#     # get flowcell
#     exp = '.*\/[\w-]+_(\d+)(?:_t\d+)?\.fastq(?:.gz)?'
#     df['flowcell'] = df.fname.str.extract(exp)

#     # check to make sure the same file stem isn't there more than once
#     # (can happen if different flow cells needed different amounts of chopping)
#     # df['file_stem'] = df.basename.str.rsplit('_', n=1, expand=True)[0]
#     exp = '.*\/([\w-]+_\d+)(?:_t\d+)?\.fastq(?:.gz)?'
#     df['file_stem'] = df.fname.str.extract(exp)
#     df['chop_num'] = df.basename.str.rsplit('.fastq', expand=True)[0].str.rsplit('_t', expand=True)[1].astype(float)
#     if df.file_stem.duplicated().any():
#         dupe_stems = df.loc[df.file_stem.duplicated(keep=False), 'basename'].tolist()
#         if not auto_dedupe:
#             raise ValueError(f'Files {dupe_stems} seem to be duplicated. Check config file.')
#         else:
#             print(f'Files {dupe_stems} seem to be duplicated. Automatically removing lower chop numbers')
#             df = df.sort_values(by='chop_num', ascending=False)
#             df = df.drop_duplicates(subset='file_stem', keep='first')

#     # extract the sample name (?)
#     temp = df.basename.str.split('_', expand=True)[[0,1]]#.str.join('_')
#     df['sample_temp'] = temp[0]+'_'+temp[1]

#     # get tech rep numbers -- each mouse has multiple reps
#     # and are therefore technical reps
#     df['techrep_num'] = df.sort_values(['genotype', 'sample_temp'],
#     							ascending=[True, True])\
#     							.groupby(['sample_temp']) \
#     							.cumcount() + 1

#     # get biorep numbers -- each sample is a different mouse
#     # and therefore a different biorep
#     temp = df[['genotype', 'sample_temp']].drop_duplicates()
#     temp.reset_index(inplace=True, drop=True)
#     temp['biorep_num'] = temp.sort_values(['genotype', 'sample_temp'],
#     							ascending=[True, True])\
#     							.groupby(['genotype']) \
#     							.cumcount()+1
#     df = df.merge(temp, how='left',
#                   on=['genotype', 'sample_temp'])
    
#     # get mouse number

#     # sample should be the genotype + mouse id
#     # so genotype + biorep
#     df['sample'] = df.genotype+'_'+df.biorep_num.astype(str)

#     # dataset should be genotype + mouse id + tech rep
#     df['dataset'] = df.genotype+'_'+df.biorep_num.astype(str)+'_'+df.techrep_num.astype(str)

#     # get the talon run number these will go into
#     talon_run_num = 0
#     df['talon_run_num'] = np.nan
#     for ind, entry, in df.iterrows():
#         if ind % datasets_per_run == 0:
#             talon_run_num += 1
#         df.loc[ind, 'talon_run_num'] = talon_run_num
#     df['talon_run_num'] = df.talon_run_num.astype(int)

#     return df

In [5]:
df = parse_config_file(config_file,
                      7,
                      auto_dedupe=True)

In [6]:
df.head()

,path,basename,genotype,batch,platform,fname,flowcell,file_stem,chop_num,sample_temp,techrep_num,biorep_num,sample,dataset,talon_run_num
0,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-blk_2_t1.fastq.gz,5xFADWT,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10721_lig-blk_2,1.0,ad002_10721,1,1,5xFADWT_1,5xFADWT_1_1,1
1,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-ss_blk_1_t1.fastq.gz,5xFADWT,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10721_lig-ss_blk_1,1.0,ad002_10721,2,1,5xFADWT_1,5xFADWT_1_2,1
2,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-blk_2_t1.fastq.gz,5xFADHEMI,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10725_lig-blk_2,1.0,ad002_10725,1,1,5xFADHEMI_1,5xFADHEMI_1_1,1
3,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-ss_blk_1_t1.fastq.gz,5xFADHEMI,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10725_lig-ss_blk_1,1.0,ad002_10725,2,1,5xFADHEMI_1,5xFADHEMI_1_2,1
4,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,5xFADHEMI,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,1,2,5xFADHEMI_2,5xFADHEMI_2_1,1


In [3]:
# get all genotype pairs
genotypes = ['wt', '5xfad', '5xhemi']

In [8]:
list(itertools.product(genotypes, genotypes))
list(itertools.combinations(genotypes, 2))

[('wt', '5xfad'), ('wt', '5xhemi'), ('5xfad', '5xhemi')]

In [4]:
def get_pairs(*lists):
    pairs = []
    for t in itertools.combinations(lists, 2):
        for pair in itertools.product(*t):
            pairs.append(pair)
    return pairs

In [5]:
get_pairs([genotypes, genotypes])

[]

In [114]:
fname = '230429_config.tsv'
auto_dedupe = True
datasets_per_run = 7
df = parse_config_file(fname, datasets_per_run, auto_dedupe)


In [115]:
df

,path,basename,genotype,dataset,sample,batch,platform,fname,flowcell,file_stem,chop_num,sample_temp,techrep_num,biorep_num,talon_run_num
0,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-blk_2_t1.fastq.gz,5xFADWT,5xFADWT_1_1,5xFADWT_1,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10721_lig-blk_2,1.0,ad002_10721,1,1,1
1,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-ss_blk_1_t1.fastq.gz,5xFADWT,5xFADWT_1_2,5xFADWT_1,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10721_lig-ss_blk_1,1.0,ad002_10721,2,1,1
2,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-blk_2_t1.fastq.gz,5xFADHEMI,5xFADHEMI_1_1,5xFADHEMI_1,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad002_10725_lig-blk_2,1.0,ad002_10725,1,1,1
3,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-ss_blk_1_t1.fastq.gz,5xFADHEMI,5xFADHEMI_1_2,5xFADHEMI_1,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad002_10725_lig-ss_blk_1,1.0,ad002_10725,2,1,1
4,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,5xFADHEMI,5xFADHEMI_2_1,5xFADHEMI_2,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11616_lig-blk_1,1.0,ad003_11616,1,2,1
5,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_2_t1.fastq,5xFADHEMI,5xFADHEMI_2_2,5xFADHEMI_2,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11616_lig-blk_2,1.0,ad003_11616,2,2,1
6,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,5xFADHEMI,5xFADHEMI_3_1,5xFADHEMI_3,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11617_lig-blk_1,1.0,ad003_11617,1,3,1
7,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_2_t1.fastq,5xFADHEMI,5xFADHEMI_3_2,5xFADHEMI_3,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11617_lig-blk_2,1.0,ad003_11617,2,3,2
8,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_1_t2.fastq,5xFADHEMI,5xFADHEMI_4_1,5xFADHEMI_4,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1,ad003_11625_lig-blk_1,2.0,ad003_11625,1,4,2
9,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_2_t1.fastq,5xFADHEMI,5xFADHEMI_4_2,5xFADHEMI_4,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2,ad003_11625_lig-blk_2,1.0,ad003_11625,2,4,2


In [113]:
def parse_config_file(fname,
                      datasets_per_run,
                      auto_dedupe=True):

    df = pd.read_csv(fname, sep='\t')

    # get flowcell
    exp = '.*\/[\w-]+_(\d+)(?:_t\d+)?\.fastq(?:.gz)?'
    df['flowcell'] = df.fname.str.extract(exp)

    # check to make sure the same file stem isn't there more than once
    # (can happen if different flow cells needed different amounts of chopping)
    # df['file_stem'] = df.basename.str.rsplit('_', n=1, expand=True)[0]
    exp = '.*\/([\w-]+_\d+)(?:_t\d+)?\.fastq(?:.gz)?'
    df['file_stem'] = df.fname.str.extract(exp)
    df['chop_num'] = df.basename.str.rsplit('.fastq', expand=True)[0].str.rsplit('_t', expand=True)[1].astype(float)
    if df.file_stem.duplicated().any():
        dupe_stems = df.loc[df.file_stem.duplicated(keep=False), 'basename'].tolist()
        if not auto_dedupe:
            raise ValueError(f'Files {dupe_stems} seem to be duplicated. Check config file.')
        else:
            print(f'Files {dupe_stems} seem to be duplicated. Automatically removing lower chop numbers')
            df = df.sort_values(by='chop_num', ascending=False)
            df = df.drop_duplicates(subset='file_stem', keep='first')

    # extract the sample name (?)
    temp = df.basename.str.split('_', expand=True)[[0,1]]#.str.join('_')
    df['sample_temp'] = temp[0]+'_'+temp[1]

    # get tech rep numbers -- each mouse has multiple reps
    # and are therefore technical reps
    df['techrep_num'] = df.sort_values(['genotype', 'sample_temp'],
    							ascending=[True, True])\
    							.groupby(['sample_temp']) \
    							.cumcount() + 1

    # get biorep numbers -- each sample is a different mouse
    # and therefore a different biorep
    temp = df[['genotype', 'sample_temp']].drop_duplicates()
    temp.reset_index(inplace=True, drop=True)
    temp['biorep_num'] = temp.sort_values(['genotype', 'sample_temp'],
    							ascending=[True, True])\
    							.groupby(['genotype']) \
    							.cumcount()+1
    df = df.merge(temp, how='left',
                  on=['genotype', 'sample_temp'])

    # sample should be the genotype + mouse id
    # so genotype + biorep
    df['sample'] = df.genotype+'_'+df.biorep_num.astype(str)

    # dataset should be genotype + mouse id + tech rep
    df['dataset'] = df.genotype+'_'+df.biorep_num.astype(str)+'_'+df.techrep_num.astype(str)

    # get the talon run number these will go into
    talon_run_num = 0
    df['talon_run_num'] = np.nan
    for ind, entry, in df.iterrows():
        if ind % datasets_per_run == 0:
            talon_run_num += 1
        df.loc[ind, 'talon_run_num'] = talon_run_num
    df['talon_run_num'] = df.talon_run_num.astype(int)

    return df

In [112]:
df

,path,basename,genotype,dataset,sample,batch,platform,fname,talon_run_num
0,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-blk_2_t1.fastq.gz,5xFADWT,NaN,NaN,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1
1,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10721_lig-ss_blk_1_t1.fastq.gz,5xFADWT,NaN,NaN,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1
2,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-blk_2_t1.fastq.gz,5xFADHEMI,NaN,NaN,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1
3,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad002_10725_lig-ss_blk_1_t1.fastq.gz,5xFADHEMI,NaN,NaN,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1
4,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_1_t1.fastq,5xFADHEMI,NaN,NaN,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1
5,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11616_lig-blk_2_t1.fastq,5xFADHEMI,NaN,NaN,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1
6,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_1_t1.fastq,5xFADHEMI,NaN,NaN,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,1
7,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11617_lig-blk_2_t1.fastq,5xFADHEMI,NaN,NaN,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2
8,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_1_t2.fastq,5xFADHEMI,NaN,NaN,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2
9,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,ad003_11625_lig-blk_2_t1.fastq,5xFADHEMI,NaN,NaN,230429,ONT,/share/crsp/lab/seyedam/share/ad_nanopore_tmp/...,2


In [108]:
df = pd.read_csv('230429_config.tsv', sep='\t')

# get flowcell 
exp = '.*\/[\w-]+_(\d+)(?:_t\d+)?\.fastq(?:.gz)?'
df['flowcell'] = df.fname.str.extract(exp)

# check to make sure the same file stem isn't there more than once 
# (can happen if different flow cells needed different amounts of chopping)
# df['file_stem'] = df.basename.str.rsplit('_', n=1, expand=True)[0]
exp = '.*\/([\w-]+_\d+)(?:_t\d+)?\.fastq(?:.gz)?'
df['file_stem'] = df.fname.str.extract(exp)
df['chop_num'] = df.basename.str.rsplit('.fastq', expand=True)[0].str.rsplit('_t', expand=True)[1].astype(float)
if df.file_stem.duplicated().any():
    dupe_stems = df.loc[df.file_stem.duplicated(keep=False), 'basename'].tolist()    
    if not auto_dedupe:
        raise ValueError(f'Files {dupe_stems} seem to be duplicated. Check config file.')
    else:
        print(f'Files {dupe_stems} seem to be duplicated. Automatically removing lower chop numbers')
        df = df.sort_values(by='chop_num', ascending=False)
        df = df.drop_duplicates(subset='file_stem', keep='first')

# extract the sample name (?)
temp = df.basename.str.split('_', expand=True)[[0,1]]#.str.join('_')
df['sample_temp'] = temp[0]+'_'+temp[1]

# get tech rep numbers -- each mouse has multiple reps 
# and are therefore technical reps
df['techrep_num'] = df.sort_values(['genotype', 'sample_temp'],
							ascending=[True, True])\
							.groupby(['sample_temp']) \
							.cumcount() + 1

# get biorep numbers -- each sample is a different mouse
# and therefore a different biorep
temp = df[['genotype', 'sample_temp']].drop_duplicates()
temp.reset_index(inplace=True, drop=True)
temp['biorep_num'] = temp.sort_values(['genotype', 'sample_temp'],
							ascending=[True, True])\
							.groupby(['genotype']) \
							.cumcount()+1
df = df.merge(temp, how='left',
              on=['genotype', 'sample_temp'])

# sample should be the genotype + mouse id 
# so genotype + biorep
df['sample'] = df.genotype+'_'+df.biorep_num.astype(str)

# dataset should be genotype + mouse id + tech rep 
df['dataset'] = df.genotype+'_'+df.biorep_num.astype(str)+'_'+df.techrep_num.astype(str)